In [ ]:
import json
import numpy as np
import pandas as pd
import re
from collections import Counter

In [ ]:
pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

2023-12-02 16:52:48.585634: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 16:52:48.585692: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 16:52:48.585722: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-02 16:52:48.643413: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 16:52:51.108851: W tensorflow/compiler/

In [ ]:
!python -m spacy download fr_core_news_sm

2023-12-02 16:53:02.917580: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 16:53:02.917643: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 16:53:02.917679: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-02 16:53:02.924958: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 16:53:04.100781: W tensorflow/compiler/

In [ ]:
import spacy
nlp_en = spacy.load("en_core_web_sm")

In [ ]:
import spacy
nlp_fr = spacy.load("fr_core_news_sm")

In [ ]:
def extract_word(text):
    match = re.search(r'<define>(.*?)<\/define>', text)
    return match.group(1) if match else None

In [ ]:
ar = nlp_en.get_pipe('attribute_ruler')

ar.add([[{"TEXT":"'ve"}]],{"LEMMA":"have"})
ar.add([[{"TEXT":"'m"}]],{"LEMMA":"be"})
ar.add([[{"TEXT":"n't"}]],{"LEMMA":"not"})
ar.add([[{"TEXT":"'ll"}]],{"LEMMA":"will"})
ar.add([[{"TEXT":"'re"}]],{"LEMMA":"be"})

In [ ]:
with open("val.model-agnostic.json", "r") as f:
    trial_data = json.load(f)

df = pd.json_normalize(trial_data)

In [ ]:
pos_entities_hyp_list = []
pos_entities_tgt_list = []

for i in range(0, len(df)):
  if df.at[i, 'task'] == 'MT':
    df.at[i, 'src'] = " ".join(token.lemma_ for token in nlp_fr(df.iloc[i]['src']) if not token.is_punct)
    src = nlp_fr(df.at[i, 'src'])
  elif df.at[i, 'task'] == 'PG':
    df.at[i, 'src'] = " ".join(token.lemma_ for token in nlp_fr(df.iloc[i]['src']) if not token.is_punct)
    src = nlp_en(df.at[i, 'tgt'])

  df.at[i, 'hyp'] = " ".join(token.lemma_ for token in nlp_en(df.iloc[i]['hyp']) if not token.is_punct)
  df.at[i, 'tgt'] = " ".join(token.lemma_ for token in nlp_en(df.iloc[i]['tgt']) if not token.is_punct)

  hyp = nlp_en(df.at[i, 'hyp'])
  tgt = nlp_en(df.at[i, 'tgt'])

  pos_entities_tgt = []
  pos_entities_hyp = []

  for token in hyp:
    pos_entities_hyp.append(token.pos_ if not token.ent_type_ else token.ent_type_)

  pos_entities_hyp_list.append(pos_entities_hyp)

  for token in tgt:
    pos_entities_tgt.append(token.pos_ if not token.ent_type_ else token.ent_type_)

  pos_entities_tgt_list.append(pos_entities_tgt)

  if df.at[i, 'task'] == 'DM':
     df.at[i, 'word_to_define'] = extract_word(df.at[i, 'src'])

  #Feature Extraction

  hyp_tokens = [token.text for token in hyp]
  tgt_tokens = [token.text for token in tgt]

  # Find the mutual words
  intersection = dict(Counter(hyp_tokens) & Counter(tgt_tokens))
  mutual_words = list(intersection.keys())

  df.at[i, 'num_of_common'] = len(mutual_words)

  df.at[i, 'word_share_ratio'] = (len(mutual_words))/(len(hyp)+len(tgt))



df['pos_entities_hyp'] = pos_entities_hyp_list
df['pos_entities_tgt'] = pos_entities_tgt_list

In [ ]:
df.head()

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination),word_to_define,num_of_common,word_share_ratio,pos_entities_hyp,pos_entities_tgt
0,resembling or characteristic of a weasel,tgt,The writer had just entered into his eighteent...,resemble a weasel in appearance,,DM,"[Hallucination, Not Hallucination, Not Halluci...",Not Hallucination,0.2,weaselly,2.0,0.181818,"[VERB, CCONJ, ADJ, ADP, DET, NOUN]","[VERB, DET, NOUN, ADP, NOUN]"
1,alternative form of sheath knife,tgt,Sailors ' and fishermen 's <define> sheath - k...,,,DM,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,0.8,sheath - knives,0.0,0.000000,"[ADJ, NOUN, ADP, ADJ, NOUN]",[]
2,obsolete a short period of time,tgt,"As to age , Bead could not form any clear impr...",poetic an instant a short moment,,DM,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0,eyewink,2.0,0.166667,"[VERB, DET, ADJ, NOUN, ADP, NOUN]","[VERB, DET, NOUN, DET, ADJ, NOUN]"
3,slang an incel,tgt,Because redpillers are usually normies or <def...,incel slang a man of a slightly lower rank on ...,,DM,"[Not Hallucination, Not Hallucination, Halluci...",Not Hallucination,0.2,Chadlites,2.0,0.100000,"[VERB, DET, NOUN]","[PROPN, PROPN, DET, NOUN, ADP, DET, ADV, ADJ, ..."
4,an island in Lienchiang County Taiwan,tgt,On the second day of massive live - fire drill...,an island in Dongyin Lienchiang Taiwan in the ...,,DM,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0,Xiyin,5.0,0.312500,"[DET, NOUN, ADP, GPE, GPE, GPE]","[DET, NOUN, ADP, FAC, FAC, GPE, ADP, LOC, LOC,..."


In [ ]:
df.to_json(r'processed_binary_df.json', orient='records')

In [ ]:
#with open("val.model-agnostic.json", "r") as f:
#    trial_data = json.load(f)
#
#df = pd.json_normalize(trial_data)

In [ ]:
for i in range(0, len(df)):

  if df.at[i, 'task'] == 'DM':
     df.at[i, 'word_to_define'] = extract_word(df.at[i, 'src'])

  #Correctness
  if df.at[i, 'label'] == 'Hallucination':
    df.at[i, 'is_hallucination'] = int(1)
  else:
    df.at[i, 'is_hallucination'] = int(0)

  hyp = nlp_en(df.at[i, 'hyp'])
  tgt = nlp_en(df.at[i, 'tgt'])

  #NER
  for token in hyp:
    if token.ent_type_:
      df.at[i, 'named_entities'] = int(1)
    else:
      df.at[i, 'named_entities'] = int(0)

  #Strong/Full
  if df.at[i, 'p(Hallucination)'] == 1:
    df.at[i, 'full_hallu'] = int(1)
    df.at[i, 'strong_hallu'] = int(0)
  elif df.at[i, 'p(Hallucination)'] > 0.6 and df.at[i, 'p(Hallucination)'] < 1:
    df.at[i, 'strong_hallu'] = int(1)
    df.at[i, 'full_hallu'] = int(0)
  else:
    df.at[i, 'strong_hallu'] = int(0)
    df.at[i, 'full_hallu'] = int(0)



columns_to_drop = ['label',	'p(Hallucination)' , 'ref','model','labels']
df = df.drop(columns=columns_to_drop)


In [ ]:
#df.head()

,hyp,src,tgt,task,word_to_define,is_hallucination,named_entities,strong_hallu,full_hallu
0,Resembling or characteristic of a weasel.,The writer had just entered into his eighteent...,Resembling a weasel (in appearance).,DM,weaselly,0.0,0.0,0.0,0.0
1,Alternative form of sheath knife,Sailors ' and fishermen 's <define> sheath - k...,.,DM,sheath - knives,1.0,0.0,1.0,0.0
2,(obsolete) A short period of time.,"As to age , Bead could not form any clear impr...","(poetic) An instant, a short moment.",DM,eyewink,0.0,0.0,0.0,0.0
3,(slang) An incel.,Because redpillers are usually normies or <def...,"(incel, _, slang) A man of a slightly lower ra...",DM,Chadlites,0.0,0.0,0.0,0.0
4,"An island in Lienchiang County, Taiwan.",On the second day of massive live - fire drill...,"An island in Dongyin, Lienchiang, Taiwan, in t...",DM,Xiyin,0.0,0.0,0.0,0.0


In [ ]:
#df.to_json(r'binary_df.json', orient='records')